<a href="https://colab.research.google.com/github/falahamro/Moral-Foundations-Analysis/blob/main/Mapping_Moral_Foundations_of_Republicans_and_Democrats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import nltk
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

import plotly.offline as py
import plotly.graph_objs as go
import plotly.figure_factory as ff

In [2]:
# Load the data: speech transcripts from 2016 US Presidential campaigns

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
campaign = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Moral Foundations Application/Mapping Morals of Republicans and Democrats/campaign_2016.csv", index_col=0)

In [4]:
campaign.head()

,Candidate,Party,Type,Date,Title,Text
0,Jeb Bush,R,speech,"June 15, 2015",Remarks Announcing Candidacy for President at ...,Thank you all very much. I always feel welcome...
1,Jeb Bush,R,speech,"July 30, 2015",Remarks to the National Urban League Conferenc...,Thank you all very much. I appreciate your hos...
2,Jeb Bush,R,speech,"August 11, 2015",Remarks at the Ronald Reagan Presidential Libr...,Thank you very much. It's good to be with all ...
3,Jeb Bush,R,speech,"September 9, 2015","Remarks in Garner, North Carolina",Thank you very much. I appreciate your hospita...
4,Jeb Bush,R,speech,"November 2, 2015","Remarks in Tampa, Florida",Thank you. It's great to be in Tampa with so m...


In [5]:
# get basic summary statistics about the data

# We first look at the unique candidates

campaign['Candidate'].unique()

array(['Jeb Bush', 'Ben Carson', 'Lincoln Chafee', 'Chris Christie',
       'Hillary Clinton ', 'Ted Cruz', 'Carly Fiorina', 'Lindsey Graham',
       'Mike Huckabee', 'Bobby Jindal', 'John Kasich', "Martin O'Malley",
       'George Pataki', 'Rand Paul', 'Rick Perry', 'Marco Rubio',
       'Bernie Sanders', 'Rick Santorum', 'Donald Trump', 'Scott Walker',
       'Jim Webb'], dtype=object)

In [6]:
# Look at the duplicated texts

campaign['Text'].value_counts()[campaign['Text'].value_counts() > 1]

To view this video please enable JavaScript, and consider upgrading to a web browser thatsupports HTML5 video                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

In [7]:
# Find out if there are null values

campaign.isnull().sum()

Candidate      0
Party          0
Type           0
Date           0
Title          0
Text         124
dtype: int64

In [8]:
campaign[campaign.isnull().any(axis=1)]

,Candidate,Party,Type,Date,Title,Text
10,Jeb Bush,R,press release,"June 15, 2015",Press Release - I Know We Can Fix This,NaN
11,Jeb Bush,R,press release,"June 15, 2015",Press Release - I Will Run To Win,NaN
12,Jeb Bush,R,press release,"June 16, 2015",Press Release - Best Part of Announcement Day,NaN
13,Jeb Bush,R,press release,"June 16, 2015",Press Release - #JebNoFilter: New Hampshire Ed...,NaN
14,Jeb Bush,R,press release,"June 16, 2015",Press Release - Preparing,NaN
...,...,...,...,...,...,...
6875,Rick Santorum,R,press release,"March 7, 2012",Press Release - An Interview with Karen Santorum,NaN
7161,Donald Trump,R,press release,"November 9, 2015",Press Release - Donald Trump Monologue - SNL,NaN
7230,Donald Trump,R,press release,"January 31, 2016",Press Release - Ivanka Trump - Find Your Iowa ...,NaN
7248,Donald Trump,R,press release,"February 20, 2016",Press Release - Donald J. Trump South Carolina...,NaN


In [9]:
campaign.describe()

,Candidate,Party,Type,Date,Title,Text
count,7485,7485,7485,7485,7485,7361
unique,21,2,3,1198,7320,7328
top,Hillary Clinton,R,press release,"February 29, 2016",Statement from Hillary Clinton,"To view this video please enable JavaScript, a..."
freq,2504,4205,6214,34,18,10


OBSERVATIONS: 



1.   There are 7485 records but only 7361 texts, and only 7328 unique texts. We would expect that every record should have a unique text. This implies there are null or duplicate values.
2.   A few texts are duplicated (included as both press releases and statements), which could give an erroneous view of the content or frequency of a candidate's speech.


1.   There are 124 null-valued texts. From the titles, we can intuit that these are mostly videos. And, many of the duplicate texts also appear to be videos from their titles. This could affect text analysis: maybe politicians are more likely to talk to certain audiences or about certain topics using videos, which could skew our analysis.


As can be seen, there are a handful of null values in the 'Text' column. Null values can mess up calculations and need to be dealt with by filling them with a dummy value or deleting the row. Because the null values are for text, and because we're most interested in the text content, we'll drop the offending rows. How you deal with null values will vary on a project-by-project basis.





In [10]:
# Let's drop the null-valued rows with dropna

campaign.dropna(inplace=True)

**Feature extraction**

the number of characters per text

the number of words per text

the number of sentences per text

the average length of words per text

the average length of sentences per text

In [11]:
import nltk
import ssl

In [12]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [13]:
# we'll extract features from text data


campaign['clean_text'] = campaign['Text'].str.lower()
campaign['words'] = campaign['clean_text'].apply(nltk.word_tokenize)
campaign['sentences'] = campaign['clean_text'].apply(nltk.sent_tokenize)

In [14]:
campaign.head()

,Candidate,Party,Type,Date,Title,Text,clean_text,words,sentences
0,Jeb Bush,R,speech,"June 15, 2015",Remarks Announcing Candidacy for President at ...,Thank you all very much. I always feel welcome...,thank you all very much. i always feel welcome...,"[thank, you, all, very, much, ., i, always, fe...","[thank you all very much., i always feel welco..."
1,Jeb Bush,R,speech,"July 30, 2015",Remarks to the National Urban League Conferenc...,Thank you all very much. I appreciate your hos...,thank you all very much. i appreciate your hos...,"[thank, you, all, very, much, ., i, appreciate...","[thank you all very much., i appreciate your h..."
2,Jeb Bush,R,speech,"August 11, 2015",Remarks at the Ronald Reagan Presidential Libr...,Thank you very much. It's good to be with all ...,thank you very much. it's good to be with all ...,"[thank, you, very, much, ., it, 's, good, to, ...","[thank you very much., it's good to be with al..."
3,Jeb Bush,R,speech,"September 9, 2015","Remarks in Garner, North Carolina",Thank you very much. I appreciate your hospita...,thank you very much. i appreciate your hospita...,"[thank, you, very, much, ., i, appreciate, you...","[thank you very much., i appreciate your hospi..."
4,Jeb Bush,R,speech,"November 2, 2015","Remarks in Tampa, Florida",Thank you. It's great to be in Tampa with so m...,thank you. it's great to be in tampa with so m...,"[thank, you, ., it, 's, great, to, be, in, tam...","[thank you., it's great to be in tampa with so..."


In [15]:
# We calculate the len on each column that has text
campaign['char_count'] = campaign['clean_text'].str.len()
campaign['word_count'] = campaign['words'].str.len()
campaign['sentence_count'] = campaign['sentences'].str.len()
campaign['avg_word_length'] = campaign['char_count'] / campaign['word_count']
campaign['avg_sentence_length'] = campaign['word_count'] / campaign['sentence_count']

In [16]:
campaign

,Candidate,Party,Type,Date,Title,Text,clean_text,words,sentences,char_count,word_count,sentence_count,avg_word_length,avg_sentence_length
0,Jeb Bush,R,speech,"June 15, 2015",Remarks Announcing Candidacy for President at ...,Thank you all very much. I always feel welcome...,thank you all very much. i always feel welcome...,"[thank, you, all, very, much, ., i, always, fe...","[thank you all very much., i always feel welco...",12383,2338,47,5.296407,49.744681
1,Jeb Bush,R,speech,"July 30, 2015",Remarks to the National Urban League Conferenc...,Thank you all very much. I appreciate your hos...,thank you all very much. i appreciate your hos...,"[thank, you, all, very, much, ., i, appreciate...","[thank you all very much., i appreciate your h...",14457,2867,126,5.042553,22.753968
2,Jeb Bush,R,speech,"August 11, 2015",Remarks at the Ronald Reagan Presidential Libr...,Thank you very much. It's good to be with all ...,thank you very much. it's good to be with all ...,"[thank, you, very, much, ., it, 's, good, to, ...","[thank you very much., it's good to be with al...",21283,4061,132,5.240827,30.765152
3,Jeb Bush,R,speech,"September 9, 2015","Remarks in Garner, North Carolina",Thank you very much. I appreciate your hospita...,thank you very much. i appreciate your hospita...,"[thank, you, very, much, ., i, appreciate, you...","[thank you very much., i appreciate your hospi...",10049,2030,92,4.950246,22.065217
4,Jeb Bush,R,speech,"November 2, 2015","Remarks in Tampa, Florida",Thank you. It's great to be in Tampa with so m...,thank you. it's great to be in tampa with so m...,"[thank, you, ., it, 's, great, to, be, in, tam...","[thank you., it's great to be in tampa with so...",14189,2564,37,5.533931,69.297297
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7480,Scott Walker,R,statement,"September 11, 2015",Statement by Governor Walker on the 14th Anniv...,"Milwaukee, Wis. -- Today, Gov. Scott Walker re...","milwaukee, wis. -- today, gov. scott walker re...","[milwaukee, ,, wis., --, today, ,, gov, ., sco...","[milwaukee, wis. -- today, gov., scott walker ...",928,170,7,5.458824,24.285714
7481,Scott Walker,R,statement,"September 11, 2015",Statement by Governor Walker on the Three-Year...,"Milwaukee, Wis. -- Gov. Scott Walker issued th...","milwaukee, wis. -- gov. scott walker issued th...","[milwaukee, ,, wis., --, gov, ., scott, walker...","[milwaukee, wis. -- gov., scott walker issued ...",733,135,6,5.429630,22.500000
7482,Jim Webb,D,speech,"September 24, 2015","Remarks in RFD-TVs ""Rural Town Hall"" West Des ...",OPPOLD:And from the beautiful Stine's Family B...,oppold:and from the beautiful stine's family b...,"[oppold, :, and, from, the, beautiful, stine, ...",[oppold:and from the beautiful stine's family ...,47284,9679,12,4.885215,806.583333
7483,Jim Webb,D,statement,"July 2, 2015",Statement Announcing Candidacy for President,"Dear friends:After many months of thought, del...","dear friends:after many months of thought, del...","[dear, friends, :, after, many, months, of, th...","[dear friends:after many months of thought, de...",11944,2208,52,5.409420,42.461538


In [17]:
# Let's convert the Date colun to type datetime

campaign['datetime'] = pd.to_datetime(campaign['Date'])

campaign.head()

,Candidate,Party,Type,Date,Title,Text,clean_text,words,sentences,char_count,word_count,sentence_count,avg_word_length,avg_sentence_length,datetime
0,Jeb Bush,R,speech,"June 15, 2015",Remarks Announcing Candidacy for President at ...,Thank you all very much. I always feel welcome...,thank you all very much. i always feel welcome...,"[thank, you, all, very, much, ., i, always, fe...","[thank you all very much., i always feel welco...",12383,2338,47,5.296407,49.744681,2015-06-15
1,Jeb Bush,R,speech,"July 30, 2015",Remarks to the National Urban League Conferenc...,Thank you all very much. I appreciate your hos...,thank you all very much. i appreciate your hos...,"[thank, you, all, very, much, ., i, appreciate...","[thank you all very much., i appreciate your h...",14457,2867,126,5.042553,22.753968,2015-07-30
2,Jeb Bush,R,speech,"August 11, 2015",Remarks at the Ronald Reagan Presidential Libr...,Thank you very much. It's good to be with all ...,thank you very much. it's good to be with all ...,"[thank, you, very, much, ., it, 's, good, to, ...","[thank you very much., it's good to be with al...",21283,4061,132,5.240827,30.765152,2015-08-11
3,Jeb Bush,R,speech,"September 9, 2015","Remarks in Garner, North Carolina",Thank you very much. I appreciate your hospita...,thank you very much. i appreciate your hospita...,"[thank, you, very, much, ., i, appreciate, you...","[thank you very much., i appreciate your hospi...",10049,2030,92,4.950246,22.065217,2015-09-09
4,Jeb Bush,R,speech,"November 2, 2015","Remarks in Tampa, Florida",Thank you. It's great to be in Tampa with so m...,thank you. it's great to be in tampa with so m...,"[thank, you, ., it, 's, great, to, be, in, tam...","[thank you., it's great to be in tampa with so...",14189,2564,37,5.533931,69.297297,2015-11-02


In [34]:
from nltk.stem.snowball import SnowballStemmer
import os
import re
import json

Moral Foundations Dictionary 

A quanteda dictionary object containing the Moral Foundations Dictionary, a publicly available dictionaries with information on the proportions of virtue and vice words for each foundation. The categories are harm (vice/virtue), fairness (vice/virtue), ingroup (vice/virtue), authority (vice/virtue), purity (vice/virtue) and morality (general).



In [35]:
# Let's load the dictionary into the mft_dict variable

with open('/content/drive/MyDrive/Colab Notebooks/Moral Foundations Application/Mapping Morals of Republicans and Democrats/foundations_dict.json') as json_data:
    mft_dict = json.load(json_data)

# Stem the words in your dictionary (this will help you get more matches)
stemmer = SnowballStemmer('english')

for foundation in mft_dict.keys():
    curr_words = mft_dict[foundation]
    stemmed_words = [stemmer.stem(word) for word in curr_words]
    mft_dict[foundation] = stemmed_words

In [36]:
# Stem the words in your dictionary (this will help you get more matches)
stemmer = SnowballStemmer('english')

for foundation in mft_dict.keys():
    curr_words = mft_dict[foundation]
    stemmed_words = [stemmer.stem(word) for word in curr_words]
    mft_dict[foundation] = stemmed_words

In [37]:
# Let's view the dictionary keys using the .key() function:

keys = mft_dict.keys()
list(keys)

['authority/subversion',
 'care/harm',
 'fairness/cheating',
 'liberty/oppression',
 'loyalty/betrayal',
 'sanctity/degradation']

In [38]:
# Check the entries associated with a key in brackets

mft_dict

{'authority/subversion': ['respect',
  'esteem',
  'regard',
  'subver',
  'say-so',
  'offic',
  'disrespect',
  'valu',
  'obedi',
  'assur',
  'honor',
  'disesteem',
  'agenc',
  'corrupt',
  'honour',
  'domin',
  'author',
  'observ',
  'confid',
  'defer',
  'bureau',
  'authori',
  'sure',
  'sanction'],
 'care/harm': ['hurt',
  'scath',
  'precaut',
  'concern',
  'attent',
  'damag',
  'care',
  'manag',
  'impair',
  'worri',
  'harm',
  'trauma',
  'guardianship',
  'aid',
  'tend',
  'caution',
  'forethought',
  'tutelag',
  'injuri',
  'upkeep',
  'mainten',
  'charg'],
 'fairness/cheating': ['equiti',
  'fair',
  'cuckold',
  'unsportsmanlik',
  'screw',
  'dirti',
  'candour',
  'cheat',
  'proport',
  'balanc',
  'inequ',
  'chican',
  'betray',
  'candor',
  'adult',
  'chous',
  'unsport',
  'unfair',
  'two-tim',
  'foul',
  'shaft',
  'fair-mind'],
 'liberty/oppression': ['self-direct',
  'self-suffici',
  'autonomi',
  'conquest',
  'burdensom',
  'independ',
  '

Observations: 

Note the enries in the dictionary have been stemmed. In other words, they have been reduced to their smallest meaningful root. 

In [39]:
# Let's try and check the number of times a second string appears in the first string


"Data science is the best major, says data scientist.".count('science')

1

In [40]:
# Let's try the lower function: 

"Rhetoric major says back: NEVER argue with a rhetoric student.".lower()

'rhetoric major says back: never argue with a rhetoric student.'

In [32]:
def clean_text(text):
    # remove punctuation using a regular expression (not covered in these modules)
    p = re.compile(r'[^\w\s]')
    no_punc = p.sub(' ', text)
    # convert to lowercase
    no_punc_lower = no_punc.lower()
    # split into individual words
    clean = no_punc_lower.split()
    return clean
    
campaign['clean_speech'] = [clean_text(s) for s in campaign['Text']]

campaign.head()

,Candidate,Party,Type,Date,Title,Text,clean_text,words,sentences,char_count,word_count,sentence_count,avg_word_length,avg_sentence_length,datetime,clean_speech,total_words
0,Jeb Bush,R,speech,"June 15, 2015",Remarks Announcing Candidacy for President at ...,Thank you all very much. I always feel welcome...,thank you all very much. i always feel welcome...,"[thank, you, all, very, much, ., i, always, fe...","[thank you all very much., i always feel welco...",12383,2338,47,5.296407,49.744681,2015-06-15,"[thank, you, all, very, much, i, always, feel,...",2284
1,Jeb Bush,R,speech,"July 30, 2015",Remarks to the National Urban League Conferenc...,Thank you all very much. I appreciate your hos...,thank you all very much. i appreciate your hos...,"[thank, you, all, very, much, ., i, appreciate...","[thank you all very much., i appreciate your h...",14457,2867,126,5.042553,22.753968,2015-07-30,"[thank, you, all, very, much, i, appreciate, y...",2638
2,Jeb Bush,R,speech,"August 11, 2015",Remarks at the Ronald Reagan Presidential Libr...,Thank you very much. It's good to be with all ...,thank you very much. it's good to be with all ...,"[thank, you, very, much, ., it, 's, good, to, ...","[thank you very much., it's good to be with al...",21283,4061,132,5.240827,30.765152,2015-08-11,"[thank, you, very, much, it, s, good, to, be, ...",3735
3,Jeb Bush,R,speech,"September 9, 2015","Remarks in Garner, North Carolina",Thank you very much. I appreciate your hospita...,thank you very much. i appreciate your hospita...,"[thank, you, very, much, ., i, appreciate, you...","[thank you very much., i appreciate your hospi...",10049,2030,92,4.950246,22.065217,2015-09-09,"[thank, you, very, much, i, appreciate, your, ...",1880
4,Jeb Bush,R,speech,"November 2, 2015","Remarks in Tampa, Florida",Thank you. It's great to be in Tampa with so m...,thank you. it's great to be in tampa with so m...,"[thank, you, ., it, 's, great, to, be, in, tam...","[thank you., it's great to be in tampa with so...",14189,2564,37,5.533931,69.297297,2015-11-02,"[thank, you, it, s, great, to, be, in, tampa, ...",2550


In [33]:
# create a new column called 'total_words'
campaign['total_words'] = [len(speech) for speech in campaign['clean_speech']]
campaign.head()

,Candidate,Party,Type,Date,Title,Text,clean_text,words,sentences,char_count,word_count,sentence_count,avg_word_length,avg_sentence_length,datetime,clean_speech,total_words
0,Jeb Bush,R,speech,"June 15, 2015",Remarks Announcing Candidacy for President at ...,Thank you all very much. I always feel welcome...,thank you all very much. i always feel welcome...,"[thank, you, all, very, much, ., i, always, fe...","[thank you all very much., i always feel welco...",12383,2338,47,5.296407,49.744681,2015-06-15,"[thank, you, all, very, much, i, always, feel,...",2284
1,Jeb Bush,R,speech,"July 30, 2015",Remarks to the National Urban League Conferenc...,Thank you all very much. I appreciate your hos...,thank you all very much. i appreciate your hos...,"[thank, you, all, very, much, ., i, appreciate...","[thank you all very much., i appreciate your h...",14457,2867,126,5.042553,22.753968,2015-07-30,"[thank, you, all, very, much, i, appreciate, y...",2638
2,Jeb Bush,R,speech,"August 11, 2015",Remarks at the Ronald Reagan Presidential Libr...,Thank you very much. It's good to be with all ...,thank you very much. it's good to be with all ...,"[thank, you, very, much, ., it, 's, good, to, ...","[thank you very much., it's good to be with al...",21283,4061,132,5.240827,30.765152,2015-08-11,"[thank, you, very, much, it, s, good, to, be, ...",3735
3,Jeb Bush,R,speech,"September 9, 2015","Remarks in Garner, North Carolina",Thank you very much. I appreciate your hospita...,thank you very much. i appreciate your hospita...,"[thank, you, very, much, ., i, appreciate, you...","[thank you very much., i appreciate your hospi...",10049,2030,92,4.950246,22.065217,2015-09-09,"[thank, you, very, much, i, appreciate, your, ...",1880
4,Jeb Bush,R,speech,"November 2, 2015","Remarks in Tampa, Florida",Thank you. It's great to be in Tampa with so m...,thank you. it's great to be in tampa with so m...,"[thank, you, ., it, 's, great, to, be, in, tam...","[thank you., it's great to be in tampa with so...",14189,2564,37,5.533931,69.297297,2015-11-02,"[thank, you, it, s, great, to, be, in, tampa, ...",2550


In [43]:
#Note: much of the following code is not covered in these modules. Read the comments to get a sense of what it does.

# do the following code for each foundation
for foundation in mft_dict.keys():
    # create a new, empty column
    num_match_words = np.zeros(len(campaign))
    stems = mft_dict[foundation]
    
    # do the following code for each foundation word
    for stem in stems:
        # find synonym matches
        wd_count = np.array([sum([wd.startswith(stem) for wd in speech]) for speech in campaign['clean_speech']])
        # add the number of matches to the total
        num_match_words += wd_count
        
    # create a new column for each foundation with the number of foundation words per speech
    campaign[foundation] = num_match_words

campaign.head()

,Candidate,Party,Type,Date,Title,Text,clean_text,words,sentences,char_count,...,avg_sentence_length,datetime,clean_speech,total_words,authority/subversion,care/harm,fairness/cheating,liberty/oppression,loyalty/betrayal,sanctity/degradation
0,Jeb Bush,R,speech,"June 15, 2015",Remarks Announcing Candidacy for President at ...,Thank you all very much. I always feel welcome...,thank you all very much. i always feel welcome...,"[thank, you, all, very, much, ., i, always, fe...","[thank you all very much., i always feel welco...",12383,...,49.744681,2015-06-15,"[thank, you, all, very, much, i, always, feel,...",2284,4.0,4.0,3.0,0.0,7.0,4.0
1,Jeb Bush,R,speech,"July 30, 2015",Remarks to the National Urban League Conferenc...,Thank you all very much. I appreciate your hos...,thank you all very much. i appreciate your hos...,"[thank, you, all, very, much, ., i, appreciate...","[thank you all very much., i appreciate your h...",14457,...,22.753968,2015-07-30,"[thank, you, all, very, much, i, appreciate, y...",2638,8.0,2.0,7.0,0.0,4.0,9.0
2,Jeb Bush,R,speech,"August 11, 2015",Remarks at the Ronald Reagan Presidential Libr...,Thank you very much. It's good to be with all ...,thank you very much. it's good to be with all ...,"[thank, you, very, much, ., it, 's, good, to, ...","[thank you very much., it's good to be with al...",21283,...,30.765152,2015-08-11,"[thank, you, very, much, it, s, good, to, be, ...",3735,12.0,5.0,1.0,0.0,4.0,5.0
3,Jeb Bush,R,speech,"September 9, 2015","Remarks in Garner, North Carolina",Thank you very much. I appreciate your hospita...,thank you very much. i appreciate your hospita...,"[thank, you, very, much, ., i, appreciate, you...","[thank you very much., i appreciate your hospi...",10049,...,22.065217,2015-09-09,"[thank, you, very, much, i, appreciate, your, ...",1880,3.0,1.0,1.0,0.0,1.0,4.0
4,Jeb Bush,R,speech,"November 2, 2015","Remarks in Tampa, Florida",Thank you. It's great to be in Tampa with so m...,thank you. it's great to be in tampa with so m...,"[thank, you, ., it, 's, great, to, be, in, tam...","[thank you., it's great to be in tampa with so...",14189,...,69.297297,2015-11-02,"[thank, you, it, s, great, to, be, in, tampa, ...",2550,8.0,3.0,1.0,1.0,0.0,7.0


In [45]:
# We divide the number of matched words by the number of total words and multiply by 100

for foundation in mft_dict.keys():
    campaign[foundation] = (campaign[foundation] / campaign['total_words']) * 100

campaign.head()

,Candidate,Party,Type,Date,Title,Text,clean_text,words,sentences,char_count,...,avg_sentence_length,datetime,clean_speech,total_words,authority/subversion,care/harm,fairness/cheating,liberty/oppression,loyalty/betrayal,sanctity/degradation
0,Jeb Bush,R,speech,"June 15, 2015",Remarks Announcing Candidacy for President at ...,Thank you all very much. I always feel welcome...,thank you all very much. i always feel welcome...,"[thank, you, all, very, much, ., i, always, fe...","[thank you all very much., i always feel welco...",12383,...,49.744681,2015-06-15,"[thank, you, all, very, much, i, always, feel,...",2284,0.007668,0.007668,0.005751,0.000000,0.013419,0.007668
1,Jeb Bush,R,speech,"July 30, 2015",Remarks to the National Urban League Conferenc...,Thank you all very much. I appreciate your hos...,thank you all very much. i appreciate your hos...,"[thank, you, all, very, much, ., i, appreciate...","[thank you all very much., i appreciate your h...",14457,...,22.753968,2015-07-30,"[thank, you, all, very, much, i, appreciate, y...",2638,0.011496,0.002874,0.010059,0.000000,0.005748,0.012933
2,Jeb Bush,R,speech,"August 11, 2015",Remarks at the Ronald Reagan Presidential Libr...,Thank you very much. It's good to be with all ...,thank you very much. it's good to be with all ...,"[thank, you, very, much, ., it, 's, good, to, ...","[thank you very much., it's good to be with al...",21283,...,30.765152,2015-08-11,"[thank, you, very, much, it, s, good, to, be, ...",3735,0.008602,0.003584,0.000717,0.000000,0.002867,0.003584
3,Jeb Bush,R,speech,"September 9, 2015","Remarks in Garner, North Carolina",Thank you very much. I appreciate your hospita...,thank you very much. i appreciate your hospita...,"[thank, you, very, much, ., i, appreciate, you...","[thank you very much., i appreciate your hospi...",10049,...,22.065217,2015-09-09,"[thank, you, very, much, i, appreciate, your, ...",1880,0.008488,0.002829,0.002829,0.000000,0.002829,0.011317
4,Jeb Bush,R,speech,"November 2, 2015","Remarks in Tampa, Florida",Thank you. It's great to be in Tampa with so m...,thank you. it's great to be in tampa with so m...,"[thank, you, ., it, 's, great, to, be, in, tam...","[thank you., it's great to be in tampa with so...",14189,...,69.297297,2015-11-02,"[thank, you, it, s, great, to, be, in, tampa, ...",2550,0.012303,0.004614,0.001538,0.001538,0.000000,0.010765
